# T1.1 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes usados
import pandas as pd # Para pegar os dados dos arquivos
from IPython.display import display # Para mostrar mais de uma informação em uma mesma célula
import ast # Para transformar string/object em estruturas de dados (listas, dicionários, ...)
import numpy as np # Para obter o total de valores por um atributo

In [ ]:
# Carregando dados de cada um dos arquivos
DataQualisRaw = pd.read_excel('Files/Qualis CC 2013-2016.xlsx')
DataDocentesRaw = pd.read_csv('Files/docentes.csv')
DataDiscentesRaw = pd.read_csv('Files/discentes.csv')
DataProducaoRaw = pd.read_csv('Files/producao.csv')
DataTrabalhosRaw = pd.read_csv('Files/trabalhos.csv')

## Análise prévia dos dados:

In [ ]:
# Pela função info, parece que temos todos os dados disponíveis para cada coluna (sem valores nulos)
display(DataQualisRaw.info())

# Pelas 5 primeiras tuplas, os dados parecem não precisar tratamento
DataQualisRaw.head()

# Fazendo uma análise prévia dos dados, DataQualisRaw parece representar o título de papers associado a um código(?)

In [ ]:
# Pela função info, parece que temos todos os dados disponíveis para cada coluna (sem valores nulos)
display(DataDocentesRaw.info())

# Pelas 5 primeiras tuplas, os dados parecem não precisar tratamento
DataDocentesRaw.head()

# Fazendo uma análise prévia dos dados, DataDocentesRaw parece representar docentes e sua relação com a instiuição acadêmica/universidade

In [ ]:
# Pela função info, parece que temos dados faltando para algumas das colunas (há valores nulos)
display(DataDiscentesRaw.info())

# Pelas tabela, devemos precisar tratar os dados da coluna "abrev". Importante ressaltar que alguns dos dados em "orientadores" parecem representar listas de dicionários através de string/object.
DataDiscentesRaw.head()

# Fazendo uma análise prévia dos dados, DataDiscentesRaw parece representar os alunos de pós-graduação das instituições acadêmicas/universidades e todos os dados atrelados a eles.

In [ ]:
# A função info nos mostra que temos valores nulos em algumas das colunas (total de valores não-nulos menor que o total de tuplas da tabela)
display(DataProducaoRaw.info())

pd.set_option('display.max_columns', 35) # Para poder visualizar todas as colunas deste data frame

# As 5 primeiras tuplas da tabela nos mostram que pelo menos alguns dados precisam ser tratados nas colunas:
# - dict_paper_autores (os dados dessa coluna na verdade representam listas de dicionários) [VER PRÓXIMA CÉLULA]
# - paper_autores (os dados dessa coluna na verdade representam listas de strings/objects) [VER CÉLULA PRÓXIMA À SEGUINTE]
# - doi
# - periodico (talvez?)
# - ano (número fracionário para ano faz sentido? cuidado, faz sim se formos considerar como período [ex.: 2018.1 e 2018.2])
DataProducaoRaw.head()

# Fazendo uma análise prévia dos dados, parece que DataProducaoRaw lista papers publicados por integrantes (docentes?) de instituições acadêmicas/universidades, informando vários outros dados ligados a esses papers.

In [ ]:
# Vemos que uma célula da segunda coluna de DataProducaoRaw é uma string/object que representa uma lista de dicionários
display(DataProducaoRaw['dict_paper_autores'][0])

# Transformando uma dessas string/objects em uma lista de dicionários
dictionaryList = ast.literal_eval(DataProducaoRaw['dict_paper_autores'][0])

display(dictionaryList) # Lista de dicionários
display(dictionaryList[0]) # Dicionário
display(dictionaryList[0]['categoria']) # Valor atribuído à chave 'categoria'
dictionaryList[0]['nome'] # Valor atribuído à chave 'nome'

In [ ]:
# Vemos que uma célula da coluna 'paper_autores' de DataProducaoRaw é uma string/object que representa uma lista de strings/objects
display(DataProducaoRaw['paper_autores'][0])

stringList = ast.literal_eval(DataProducaoRaw['paper_autores'][0])

display(stringList) # Lista de strings/objects
display(stringList[0]) # Uma (TADAM!) string/object

In [ ]:
# Pela função info, parece que temos valores nulos em algumas das colunas
display(DataTrabalhosRaw.info())

# Pelas 5 primeiras tuplas, parece que precisaremos tratar os dados das colunas:
# - keywords
# - palavras-chave
# - paginas (número fracionário para páginas faz sentido?)
# - programa (talvez?)
DataTrabalhosRaw.head()

# Fazendo uma análise prévia dos dados, DataTrabalhosRaw parece representar os dados referentes aos trabalhos de conclusão de curso dos alunos de pós-graduação

## Limpeza dos dados:
Obs.: Esta parte pode ser retirada do trabalho, visto que podemos precisar fazer limpezas mais rigorosas para responder uma pergunta e outras menos rigorosas para responder outras perguntas com o mesmo data frame original. Talvez vale a pena manter esta área se conseguirmos fazer uma limpeza otimizada para um mesmo data frame que case bem com todas as perguntas que vão usar o data frame original em questão. Fica como algo extra a fazer no trabalho, se acabarmos com tempo sobrando...

In [ ]:
# Para DataProducaoRaw

# Vimos que a tabela "ano" possui valores nuyméricos fracionários e um valor nulo. Vemos que apenas dois:
display(DataProducaoRaw['ano'].unique())

# Vemos que há apenas uma tupla onde temos esse valor e ela na verdade está cheia de valores nulos. Vale a pena retirar ela por esse aspecto, mas podemos usar o valor de "paper_id" presente para encontrar problemas equivalentes nos outros data frames que tenham o mesmo id.
display(DataProducaoRaw.loc[DataProducaoRaw['ano'].isnull()])

# O data frame "limpo" (obs.: pode ir mudando conforme vamos respondendo as perguntas)
producaoDF = DataProducaoRaw.loc[DataProducaoRaw['ano'].isnull() == False]
producaoDF = producaoDF.reset_index(drop = True)
producaoDF.info()

# Perguntas:
## 1) Quantos professores (docentes) havia em cada instituição em 2017, em cada quadro (permanente, colaborador)?

In [ ]:
# DataDocentesRaw contém os dados necessários para responder a pergunta (colunas "ies" e "categoria")
DataDocentesRaw.head()

In [ ]:
# Verificando quais as instituições, parecem OK
display(DataDocentesRaw['ies'].unique())

# Verificando quais as categorias, parecem OK
DataDocentesRaw['categoria'].unique()

In [ ]:
# Montando nosso data frame
docentesByIESDF = pd.DataFrame(columns=('IES', 'Permanentes', 'Colaboradores'))
docentesByIESDF['IES'] = DataDocentesRaw['ies'].unique()
docentesByIESDF['Permanentes'] = 0
docentesByIESDF['Colaboradores'] = 0

# Percorrendo o data frame original para fazer a contagem dos tipos de docente por instituição
for indx in range(0, len(DataDocentesRaw)):
    if DataDocentesRaw['categoria'][indx] == 'PERMANENTE':
        docentesByIESDF.loc[docentesByIESDF['IES'] == DataDocentesRaw['ies'][indx], 'Permanentes'] += 1
    elif DataDocentesRaw['categoria'][indx] == 'COLABORADOR':
        docentesByIESDF.loc[docentesByIESDF['IES'] == DataDocentesRaw['ies'][indx], 'Colaboradores'] += 1
    else: # Caso encontremos um imprevisto
        print("\'categoria\' error! Value:", DataDocentesRaw['categoria'][indx], " Row:", indx)
        
# O resultado obtido
docentesByIESDF

## <span style="color:green">Desejável para a questão:</span>
- Mostrar um gráfico (barras na vertical?) com os valores obtidos 

## 2) Quantos alunos (discentes) de Mestrado/Doutorado havia em cada programa em 2017?

In [ ]:
# DataDiscentesRaw contém os dados necessários para responder a pergunta (colunas "nivel" e "programa")
DataDiscentesRaw.head()

In [ ]:
# Pegando apenas as colunas que queremos
discentesDFQ2 = DataDiscentesRaw[['nivel','programa']]
display(discentesDFQ2.head()) # Vemos que temos uma coluna com valores vazios

display(discentesDFQ2.info()) # Vemos que 3 valores para cada coluna estão comprometidos

display(discentesDFQ2.loc[discentesDFQ2['nivel'].isnull()]) # Felizmente vemos que os 3 valores de cada coluna fazem parte das mesmas tuplas

# Limpando o data frame e mantendo a ordenação dos índices
discentesDFQ2 = discentesDFQ2.loc[discentesDFQ2['nivel'].isnull() == False]
discentesDFQ2 = discentesDFQ2.reset_index(drop = True)
discentesDFQ2.info()

In [ ]:
# Verificando valores em "nivel", vemos que existe pelo menos um valor 'Graduação' que não nos interessa
display(discentesDFQ2['nivel'].unique())

# Verificando valores em "programa"
display(discentesDFQ2['programa'].unique())

# Eliminando valores 'Graduação', obtemos uma redução considerável de tuplas
discentesDFQ2 = discentesDFQ2.loc[discentesDFQ2['nivel'] != 'Graduação']
discentesDFQ2 = discentesDFQ2.reset_index(drop = True)
discentesDFQ2.info()

In [ ]:
# Montando nosso data frame
discentesByPrograma = pd.DataFrame(columns=('Programa', 'Mestrandos', 'Doutorandos'))
discentesByPrograma['Programa'] = discentesDFQ2['programa'].unique()
discentesByPrograma['Mestrandos'] = 0
discentesByPrograma['Doutorandos'] = 0

# Percorrendo o data frame limpo para fazer a contagem dos tipos de nível por programa
for indx in range(0, len(discentesDFQ2)):
    if discentesDFQ2['nivel'][indx] == 'Mestrado':
        discentesByPrograma.loc[discentesByPrograma['Programa'] == discentesDFQ2['programa'][indx], 'Mestrandos'] += 1
    elif discentesDFQ2['nivel'][indx] == 'Doutorado':
        discentesByPrograma.loc[discentesByPrograma['Programa'] == discentesDFQ2['programa'][indx], 'Doutorandos'] += 1
    else: # Caso encontremos um imprevisto
        print("\'nivel\' error! Value:", discentesDFQ2['nivel'][indx], " Row:", indx)
        
# O resultado obtido
discentesByPrograma

## <span style="color:green">Desejável para a questão:</span>
- Mostrar um gráfico (barras na vertical?) com os valores obtidos

## 3) Qual foi a taxa de alunos de Mestrado/Doutorado por professor do quadro permanente em cada programa em 2017?
<br/>
<br/>
Help: Ordenar lista de professores permanentes por programa (dicionário? key: programa; valor: lista) e pra cada professor dessa lista dizer quantos mestrandos e quantos doutorandos(dicionário? key: professor; valor: lista tamanho 2 com valor total de mestrando e valor total de doutorandos **OU** data frame com professor - mestrandos - doutorandos). :^)

# <span style="color:red">Solução errada!</span>
É pra mostrar (apenas) taxa Total ALunos/Total Professores para cada programa. Refazer tudo. :(

In [ ]:
# Vamos usar os data frames abaixo
display(DataDocentesRaw.head()) # Nos diz que professor é permanente
DataDiscentesRaw.head() # Nos diz qual o nível do aluno, o programa do qual fez parte e quem foi seu professor

In [ ]:
# Começando por DataDiscentesRaw
discentesDFQ3 = DataDiscentesRaw[['nivel','programa', 'orientador', 'orientadores']]
display(discentesDFQ3.info()) # Vemos que coluna 'orientador' tem muitos valores faltando e 'orientadores' tem menos, estranho
display(discentesDFQ3[0:1]) # A primeira linha nos mostra que existe um array em 'orientadores' mas este está vazio enquanto 'orientador' está com NaN, deve ser o que está causando a diferença numérica entre as colunas

# Retirando todas as colunas onde 'orientador' está nulo
discentesDFQ3 = discentesDFQ3.loc[discentesDFQ3['orientador'].isnull() == False]
discentesDFQ3 = discentesDFQ3.reset_index(drop = True)
display(discentesDFQ3.info()) # Agora todas as colunas possuem a mesma quantidade de linhas, originalmente sendo a de 'orientador' (o menor valor que tínhamos antes)

# Vemos que acabamos tirando o valor 'Graduação' em 'nivel' no meio dessa limpeza
display(discentesDFQ2['nivel'].unique())

discentesDFQ3.head()

In [ ]:
# Agora vamos com DataDocentesRaw
docentesDFQ3 = DataDocentesRaw[['categoria','nome']]
display(docentesDFQ3.info()) # Parece tudo OK

# Eliminando valores 'COLABORADOR'
docentesDFQ3 = docentesDFQ3.loc[docentesDFQ3['categoria'] != 'COLABORADOR']
docentesDFQ3 = docentesDFQ3.reset_index(drop = True)
display(docentesDFQ3.info()) # A redução não foi tão expressiva

docentesDFQ3.head()

In [ ]:
# Montando um dicionário, usando valores de coluna 'programa' como chave
q3Dictionary = {discentesDFQ3['programa'].unique()[i]: 0 for i in range(0, len(discentesDFQ3['programa'].unique()))}

# Procurando valores por valor 'programa'
for key in q3Dictionary.keys():
    # Reduzindo meu data frame 'doscentes' para o valor 'programa' da iteração
    tempDiscentesDFQ3 = discentesDFQ3.loc[discentesDFQ3['programa'] == key]
    
    # Reduzindo o data frame temporário ainda mais para valores 'orientador' com valor 'PERMANENTE'
    tempDiscentesDFQ3 = tempDiscentesDFQ3.loc[tempDiscentesDFQ3['orientador'].isin(docentesDFQ3['nome'])]
    tempDiscentesDFQ3 = tempDiscentesDFQ3.reset_index(drop = True)
    
    # Criando data frame a entrar como valor no dicionário
    discentesByOrientador = pd.DataFrame(columns=('Professor', 'Mestrandos', 'Doutorandos'))
    discentesByOrientador['Professor'] = tempDiscentesDFQ3['orientador'].unique()
    discentesByOrientador['Mestrandos'] = 0
    discentesByOrientador['Doutorandos'] = 0
    discentesByOrientador['Taxa Mestrandos'] = 0.0
    discentesByOrientador['Taxa Doutorandos'] = 0.0
    discentesByOrientador['Taxa Alunos'] = 0.0
    
    # Percorrendo o data frame temporário para fazer a contagem dos tipos de nível por orientador
    for indx in range(0, len(tempDiscentesDFQ3)):
        if tempDiscentesDFQ3['nivel'][indx] == 'Mestrado':
            discentesByOrientador.loc[discentesByOrientador['Professor'] == tempDiscentesDFQ3['orientador'][indx], 'Mestrandos'] += 1
        elif tempDiscentesDFQ3['nivel'][indx] == 'Doutorado':
            discentesByOrientador.loc[discentesByOrientador['Professor'] == tempDiscentesDFQ3['orientador'][indx], 'Doutorandos'] += 1
        else: # Caso encontremos um imprevisto
            print("\'nivel\' error! Value:", tempDiscentesDFQ3['nivel'][indx], " Row:", indx)
    
    # Calculando valores totais POR PROGRAMA
    totalMestrandos = np.sum(discentesByOrientador[['Mestrandos']].sum())
    totalDoutorandos = np.sum(discentesByOrientador[['Doutorandos']].sum())
    totalAlunos = np.sum(discentesByOrientador[['Mestrandos', 'Doutorandos']].sum())
    
    # Percorrendo o data frame temporário para calcular as taxas
    for indx in range(0, len(discentesByOrientador)):
        # O total de alunos DO PROFESSOR
        totalAlunosByProfessor = np.sum(discentesByOrientador.loc[discentesByOrientador['Professor'] == discentesByOrientador['Professor'][indx]][['Mestrandos', 'Doutorandos']].sum())
        
        # Calculo das taxas DO PROFESSOR em relação ao TOTAL DO PROGRAMA
        discentesByOrientador.loc[discentesByOrientador['Professor'] == discentesByOrientador['Professor'][indx], 'Taxa Mestrandos'] = 100.* discentesByOrientador['Mestrandos'][indx] / totalMestrandos
        discentesByOrientador.loc[discentesByOrientador['Professor'] == discentesByOrientador['Professor'][indx], 'Taxa Doutorandos'] = 100.* discentesByOrientador['Doutorandos'][indx] / totalDoutorandos
        discentesByOrientador.loc[discentesByOrientador['Professor'] == discentesByOrientador['Professor'][indx], 'Taxa Alunos'] = 100.* totalAlunosByProfessor / totalAlunos
    
    q3Dictionary[key] = discentesByOrientador # Associando data frame à respectiva chave do dicionário

# O dicionário/monstro final
q3Dictionary

In [ ]:
# Para visualizar todas as tuplas
pd.set_option('display.max_rows', None)

# Mostrando os resultados obtidos
for key in q3Dictionary.keys():
    dataFrame = q3Dictionary[key]
    print("\n\n\n\n\nPrograma: ", key)
    display(dataFrame)

In [ ]:
# Apenas de apoio. APAGAR ANTES DE ENTREGAR!!!
display(np.sum(discentesByOrientador[['Mestrandos', 'Doutorandos']].sum()))
np.sum(discentesByOrientador.loc[discentesByOrientador['Professor'] == 'EDUARDO MARQUES'][['Mestrandos', 'Doutorandos']].sum())

## <span style="color:green">Desejável para a questão:</span>
- Confirmar que dados na coluna 'orientadores' tenham a mesma informação da sua respectiva tupla na coluna 'orientador' para justificar o uso de apenas 'orientador'. Ou seja, se existe só 1 orientador por aluno. (O nome 'orientadores' leva a crer que o aluno teve mais de um orientador no projeto)
- Deixar código do dicionário menos confuso
- Ver qual gráfico pode ajudar melhor a visualizar os resultados e implementá-lo

## 4) Qual foi a distribuição de alunos de Mestrado/Doutorado pelos professores de cada programa em 2017?

Kaplan, o dicionário que fiz na questão 3 aparentemente já tem as respostas para esta questão. Eu sugiro você focar aqui em como representar as informações bem visualmente. Ou você pode apresentar uma tabela mais bonita que meu dicionário que mostre as informações contidas nele, fica a teu critério. :P

## 5) Quantos alunos de Mestrado/Doutorado defenderam suas dissertações/teses em 2017 (arquivo trabalhos.csv)?

In [ ]:
# Vemos que faltam dados em DataTrabalhosRaw
display(DataTrabalhosRaw.info())
DataTrabalhosRaw.head()

In [ ]:
# Pegando as colunas que nos interessam e eliminando valores nulos através de uma das colunas do respectivo data frame
trabalhosDFQ5 = DataTrabalhosRaw.loc[DataTrabalhosRaw['autor'].isnull() == False][['autor', 'tipo', 'data_defesa']]
trabalhosDFQ5 = trabalhosDFQ5.sort_values(['autor', 'tipo'], ascending=[True, True])
trabalhosDFQ5 = trabalhosDFQ5.reset_index(drop = True)

display(trabalhosDFQ5.head())

# Vemos que felizmente nos desfizemos de todos os valores nulos. A coluna "data_defesa" foi deixada para nos certificarmos que
# todos os discentes aqui presentes realizaram as defesas
trabalhosDFQ5.info()

In [ ]:
# Em trabalhosDFQ5 temos apenas Tese e Dissertação, como queremos
display(trabalhosDFQ5['tipo'].unique())

In [ ]:
# Calculando o total de Mestrandos e Doutorandos
quantityMestrandos = len(trabalhosDFQ5.loc[trabalhosDFQ5['tipo'] == 'DISSERTAÇÃO'])
quantityDoutorandos = len(trabalhosDFQ5.loc[trabalhosDFQ5['tipo'] == 'TESE'])

print("Total Mestrandos:", quantityMestrandos)
print("Total Doutorandos:", quantityDoutorandos)

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um gráfico de barras mostrando cada um dos valores. Uma barra para cada tipo.

## 6) Como os trabalhos de Mestrado/Doutorado defendidos em 2017 foram distribuídos pelas áreas de pesquisa dos programas?

# <span style="color:red">OBS.:</span>
Usar coluna tipo de tabela trabalhos para mestrado (tipo == 'DISSERTAÇÃO') e doutorado (tipo == 'TESE'). 

## 7) Como as defesas de Mestrado/Doutorado foram distribuídas ao longo do ano de 2017?

In [ ]:
# Para responder a pergunta, usaremos de novo 'DataTrabalhosRaw' (contém as datas) e 'DataDiscentesRaw' (contém o nivel)
display(DataTrabalhosRaw.info())
display(DataTrabalhosRaw.head())

In [ ]:
# Pegando os atributos que nos interessa, fazendo as devidas limpezas/conversões
trabalhosDFQ7 = DataTrabalhosRaw.loc[(DataTrabalhosRaw['data_defesa'].isnull() == False)][['autor', 'tipo', 'data_defesa']]
trabalhosDFQ7['data_defesa'] = pd.to_datetime(trabalhosDFQ7['data_defesa'])
trabalhosDFQ7 = trabalhosDFQ7.sort_values(['data_defesa', 'tipo'], ascending=[True, True])
trabalhosDFQ7 = trabalhosDFQ7.reset_index(drop = True)

display(trabalhosDFQ7.info())
trabalhosDFQ7.head()

In [ ]:
# Montando nosso data frame
tipoByMes = pd.DataFrame(columns=('Mes', 'Mestrandos', 'Doutorandos'))
tipoByMes['Mes'] = trabalhosDFQ7['data_defesa'].dt.month.unique()
tipoByMes['Mestrandos'] = 0
tipoByMes['Doutorandos'] = 0

# Fazendo a contagem em relação ao tipo pra cada mês
for indx in range(0, len(trabalhosDFQ7)):
    if trabalhosDFQ7['tipo'][indx] == 'DISSERTAÇÃO':
        tipoByMes.loc[tipoByMes['Mes'] == trabalhosDFQ7['data_defesa'].dt.month[indx], 'Mestrandos'] += 1
    elif trabalhosDFQ7['tipo'][indx] == 'TESE':
        tipoByMes.loc[tipoByMes['Mes'] == trabalhosDFQ7['data_defesa'].dt.month[indx], 'Doutorandos'] += 1

# Nosso data frame resultante
tipoByMes

## **<span style="color:green">Desejável para a questão:</span>**
- Montar um "gráfico de linhas" mostrando a evolução de cada tipo ao longo dos meses. Uma linha para cada tipo (mestrado e doutorado)

## **<span style="color:red">Detalhes a ficarmos atentos para o trabalho:</span>**
- Verificar repetição de dados (aparentemente temos papers duplicados)
- Estar atento a outros detalhes sobre os dados não percebidos na análise inicial dos dados
- Tentar manter a limpeza dos dados no campo de limpeza para organização, sempre que possível
    - No campo de limpeza, tentar usar uma célula para cada data frame raw, sempre que possível

In [ ]:
# Caso bizarro: matricula feita em 2009 com defesa em 2017
display(trabalhosDFQ7.loc[trabalhosDFQ7['autor'] == 'RAFAEL SOARES PADILHA'])
display(discentesDFQ7.loc[discentesDFQ7['autor'] == 'RAFAEL SOARES PADILHA'])